# To do

1. fix the problem leading to the warning "Some of the fieldIDs were not not found in self._UID_to_descrpt dictionary. Therefore their categories are not included in self.categories" 
2. use pyspark instead to speed up

# Functions
1. find_num_years_disease(newdf,FID,FID_corr,patientID)    
for index specified by patientID, Field ID specified by FID, field ID corresponding to the time event occured and a given patientID specifying the row, it will calculate the difference between event time and age at recruitment from the field ID 21022-0.0
2. tidy_coded_fieldID(bs_html_file,coding,prefix,FID,df)
if a datafield is coded, it will take the beautiful soup object pertaining to the html file (bs_html_file) and the coding of the field ID, e.g. "Coding 1001", and create a tidy table with the column names set to prefix+FID value. df is the df where the FID exists.

# Dictionaries
1. categories: a list of dictionaries in translating categories to fieldIDs in each category
2. fieldID_to_ctgry_dict: reverse of above
3. UID_to_dscrpt: field ID to description and other useful information
4. disease_code_to_disease_dict: disease codes Data coding 6 to disease name for self reported diseases

In [1]:
from IPython.display import display, HTML
import pandas as pd
from lxml import etree
import time
import re
import matplotlib.pyplot as plt
import random
import codecs
import numpy as np
import pickle
from UKBB import UKBDicts,preprocess,tidy_coded_fieldID




from bs4 import BeautifulSoup
import requests
from lxml import etree
import pandas as pd
import unicodedata

In [4]:
#!jupyter nbconvert --to script 'UKBB.ipynb'
#%run helper_functions.py

[NbConvertApp] Converting notebook UKBB.ipynb to script
[NbConvertApp] Writing 12041 bytes to UKBB.py


# Read the field ID summary html file

In [2]:
#PATH_data=r"/home/workspace/datasets/ukbiobank/ukbiobank_yajiez/dataset/ukb26438.csv"
#PATH_dict=r"/home/workspace/datasets/ukbiobank/ukbiobank_yajiez/dataset/ukb26438.html"

PATH_data=r"/home/workspace/datasets/ukbiobank/biobank_milad_34943/ukb34943.csv"
PATH_dict=r"/home/mo/ukb34943.html"

ukb=UKBDicts(PATH_data,PATH_dict,"file")

categories,UID_to_dscrpt,fieldID_to_ctgry_dict=ukb.categories,ukb._UID_to_dscrpt,ukb._fieldID_to_ctgry_dict

Loaded a dictionary with 8226 field ID:category entries
Parsed 1827 unique field IDs from the UKBiobank html file
Time elapsed for extracting unique field IDs: 7.471305480226874


/home/mo/Env_vars_DL/UKBB.py:180: UserWarning: Some of the fieldIDs were not not found in self._UID_to_descrpt dictionary. Therefore their categories are not included in self.categories. See self.fieldID_no_category             for a list of these
  warnings.warn("Some of the fieldIDs were not not found in self._UID_to_descrpt dictionary. Therefore their categories are not included in self.categories. See self.fieldID_no_category             for a list of these")


In [30]:
#delete ASAP

bs = BeautifulSoup(open(PATH_dict).read(), 'html.parser')
#df_tidy=tidy_coded_fieldID(bs,"Coding 1001","Ethnicity_","21000",df_temp)

table = bs.find(lambda tag: tag.name=='table' and tag.has_attr('summary') and tag['summary']=="Coding 4")
    
pd_coding=pd.read_html(str(table))


#pd.read_csv(r'tidy_data_tiles/summary_diagnoses.csv',nrows=1).to_csv("/home/shared/coding6.csv")

In [28]:
pd_coding[0]

[   #   Code                    Meaning   Unnamed: 3
 0  1  20003  Treatment/medication code  Medications]

# 2. Read data as pd for correctness check

## 2.1 read datafields by categories in numpy

In [10]:
my_cols=[col for col in column_names if ("41270" in col) | ("41280" in col)]

In [13]:
tic = time.perf_counter()
column_names = pd.read_csv(PATH_data, dtype='unicode',nrows=1).columns
df = pd.read_csv(PATH_data,dtype='unicode',engine='python',usecols=my_cols)
toc = time.perf_counter()
print('Time to read the data: '+str(toc-tic))

Time to read the data: 543.3999435610604


In [14]:
df.shape

(502527, 426)

In [16]:
df.to_csv(r'/home/shared/mo/UKBData/diagnosesraw.csv')

# 2.2 Read Demographics (baseline characteristics) and socioeconomics and  store 

## 2.2.1 Read Socio Demographics

In [122]:
selected_categories={'Baseline characteristics':['include','31','34','52','189','21022']}
process_config={'31':{'type':'int'},'34':{'type':'string'},'52':{'type':'string'},'189':{'type':'float'},'21022':{'type':'int'}}

newdf=preprocess(selected_categories,ukb.categories,ukb._UID_to_dscrpt,df,process_config)

"""#Next grab 20002 (self-reported non-cancer) and 20009 (age of self-reported non-cancer) 
and seperate illnesses based on the time of occurance to and time window"""

newdf['birth_date']=pd.to_datetime(newdf['34-0.0']+newdf['52-0.0'],format='%Y%m')
newdf.drop(columns=['34-0.0','52-0.0'],inplace=True)

From the category Baseline characteristics the following where selected: 
 	 {'34', '21022', '189', '52', '31'}
	 FID 34:  Year of birth
	 FID 21022:  Age at recruitment
	 FID 189:  Townsend deprivation index at recruitment
	 FID 52:  Month of birthUses data-coding 8
	 FID 31:  SexUses data-coding 9


In [123]:
selected_categories_tidy={'demographics and socioeconomics':['gender',
                                                      'birth_date',
                                                      'Townsend deprivation index at recruitment',
                                                      'Age at recruitment']}
selected_categories_tidy_columns={'gender':{"col_name":['31-0.0'],"type":"binary"},
                                  'birth_date':{"col_name":['birth_date'],"type":"date"},
                                  'Townsend deprivation index at recruitment':{"col_name":['189-0.0'],"type":"float"},
                                  'Age at recruitment':{"col_name":['21022-0.0'],"type":"int"}}
selected_categories_exclusions=[]

## 2.2.2 Read Ethinicity and add 

In [124]:
selected_categories={'Ethnicity':['include','21000']}
process_config={'21000':{'type':'string'}}

df_temp=preprocess(selected_categories,ukb.categories,ukb._UID_to_dscrpt,df,process_config)

df_temp=df_temp[['21000-0.0']]

bs = BeautifulSoup(open(PATH_dict).read(), 'html.parser')
df_tidy=tidy_coded_fieldID(bs,"Coding 1001","Ethnicity_","21000",df_temp)

newdf=pd.concat([newdf,df_tidy],axis=1)

selected_categories_tidy['demographics and socioeconomics'].append("Ethnicity")
selected_categories_tidy_columns['Ethnicity']={"col_name":list(df_tidy.columns.values),"type":"binary"}

From the category Ethnicity the following where selected: 
 	 {'21000'}
	 FID 21000:  Ethnic backgroundUses data-coding 1001
Ethnicity_African                         3394
Ethnicity_Any other Asian background      1815
Ethnicity_Any other Black background       123
Ethnicity_Any other mixed background      1033
Ethnicity_Any other white background     16334
Ethnicity_Asian or Asian British            43
Ethnicity_Bangladeshi                      236
Ethnicity_Black or Black British            27
Ethnicity_British                       442604
Ethnicity_Caribbean                       4517
Ethnicity_Chinese                         1574
Ethnicity_Do not know                      217
Ethnicity_Indian                          5951
Ethnicity_Irish                          13209
Ethnicity_Mixed                             49
Ethnicity_Other ethnic group              4558
Ethnicity_Pakistani                       1837
Ethnicity_Prefer not to answer            1662
Ethnicity_White              

## 2.2.3 Read and add socioeconomics
1. household income before tax, 
2. current employment status,
3. Townsend deprivation index (done in baseline) at recruitment, 
4. education qualifications 

In [204]:
word='household'
ukb.search_description(word)
print(ukb._fieldID_to_ctgry_dict['738'])
ukb.categories['Household']

Field ID: 709 _ Number in householdUses data-coding 100291

Field ID: 728 _ Number of vehicles in householdUses data-coding 100293

Field ID: 738 _ Average total household income before taxUses data-coding 100294

Field ID: 1259 _ Smoking/smokers in householdUses data-coding 100370

Field ID: 6141 _ How are people in household related to participantUses data-coding 100292

Field ID: 10877 _ Average total household income before tax (pilot)Uses data-coding 100657

['Average total household income before tax', 'Household']


['670: Type of accommodation lived inUses data-coding 100286',
 '680: Own or rent accommodation lived inUses data-coding 100287',
 '699: Length of time at current addressUses data-coding 100290',
 '709: Number in householdUses data-coding 100291',
 '728: Number of vehicles in householdUses data-coding 100293',
 '738: Average total household income before taxUses data-coding 100294',
 '6139: Gas or solid-fuel cooking/heatingUses data-coding 100288',
 '6140: Heating type(s) in homeUses data-coding 100289',
 '6141: How are people in household related to participantUses data-coding 100292',
 '10860: Gas or solid-fuel cooking/heating (pilot)Uses data-coding 100656',
 '10877: Average total household income before tax (pilot)Uses data-coding 100657']

In [126]:
FID='6142'
data_coding='100295'
FID_tidy_name="Current employment status"
selected_categories={ukb._fieldID_to_ctgry_dict[FID][1]:['include',FID]}
process_config={FID:{'type':'string'}}

df_temp=preprocess(selected_categories,ukb.categories,ukb._UID_to_dscrpt,df,process_config)

df_temp=df_temp[[FID+'-0.0']]

df_tidy=tidy_coded_fieldID(bs,"Coding "+data_coding,"Employment_",FID,df_temp)

newdf=pd.concat([newdf,df_tidy],axis=1)

selected_categories_tidy['demographics and socioeconomics'].append(FID_tidy_name)
selected_categories_tidy_columns[FID_tidy_name]={"col_name":list(df_tidy.columns.values),"type":"binary"}

From the category Employment the following where selected: 
 	 {'6142'}
	 FID 6142:  Current employment statusUses data-coding 100295
Employment_Doing unpaid or voluntary work                        2328
Employment_Full or part-time student                             1343
Employment_In paid employment or self-employed                 287163
Employment_Looking after home and/or family                     13859
Employment_None of the above                                     2801
Employment_Prefer not to answer                                  2082
Employment_Retired                                             166984
Employment_Unable to work because of sickness or disability     16833
Employment_Unemployed                                            8264
dtype: int64


In [127]:
selected_categories={'Household':['include','738']}
process_config={'738':{'type':'string'}}

df_temp=preprocess(selected_categories,ukb.categories,ukb._UID_to_dscrpt,df,process_config)

df_temp=df_temp[['738-0.0']]

df_tidy=tidy_coded_fieldID(bs,"Coding 100294","Household_inc_","738",df_temp)

newdf=pd.concat([newdf,df_tidy],axis=1)

selected_categories_tidy['demographics and socioeconomics'].append("Household Income")
selected_categories_tidy_columns['Household Income']={"col_name":list(df_tidy.columns.values),"type":"binary"}

From the category Household the following where selected: 
 	 {'738'}
	 FID 738:  Average total household income before taxUses data-coding 100294
Household_inc_18,000 to 30,999        108178
Household_inc_31,000 to 51,999        110775
Household_inc_52,000 to 100,000        86269
Household_inc_Do not know              21305
Household_inc_Greater than 100,000     22932
Household_inc_Less than 18,000         97207
Household_inc_Prefer not to answer     49848
dtype: int64


In [128]:
FID='6138'
data_coding='100305'
FID_tidy_name="Qualifications"
selected_categories={ukb._fieldID_to_ctgry_dict[FID][1]:['include',FID]}
process_config={FID:{'type':'string'}}

df_temp=preprocess(selected_categories,ukb.categories,ukb._UID_to_dscrpt,df,process_config)

df_temp=df_temp[[FID+'-0.0']]

df_tidy=tidy_coded_fieldID(bs,"Coding "+data_coding,"Qualifications_",FID,df_temp)

newdf=pd.concat([newdf,df_tidy],axis=1)

selected_categories_tidy['demographics and socioeconomics'].append(FID_tidy_name)
selected_categories_tidy_columns[FID_tidy_name]={"col_name":list(df_tidy.columns.values),"type":"binary"}

From the category Education the following where selected: 
 	 {'6138'}
	 FID 6138:  QualificationsUses data-coding 100305
Qualifications_A levels/AS levels or equivalent                            55325
Qualifications_CSEs or equivalent                                          26887
Qualifications_College or University degree                               161171
Qualifications_NVQ or HND or HNC or equivalent                             32730
Qualifications_None of the above                                           85275
Qualifications_O levels/GCSEs or equivalent                               105200
Qualifications_Other professional qualifications eg: nursing, teaching     25805
Qualifications_Prefer not to answer                                         5493
dtype: int64


In [129]:
newdf.to_csv(r'tidy_data_tiles/sociodem_socioeco.csv')
newdf=pd.DataFrame()

# 2.3 Anthropometrics and cardiac risk factors
1. Standing Height, 
2. BMI,
3. SBP, DBP, PR 
4. Drinking, and Smoking status

In [130]:
FID='50'
#data_coding='100261'
FID_tidy_name="Standing height"
selected_categories={fieldID_to_ctgry_dict[FID][1]:['include',FID]}
process_config={FID:{'type':'string'}}

df_temp=preprocess(selected_categories,categories,UID_to_dscrpt,df,process_config)

print(df_temp)
df_temp=df_temp[[FID+'-0.0']]

#df_tidy=tidy_coded_fieldID(bs,"Coding "+data_coding,FID_tidy_name+"_",FID,df_temp)

newdf=pd.concat([newdf,df_temp],axis=1)
df_tidy=df_temp

selected_categories_tidy[fieldID_to_ctgry_dict[FID][1]]=[FID_tidy_name]
selected_categories_tidy_columns[FID_tidy_name]={"col_name":list(df_tidy.columns.values),"type":"int"}

From the category Body size measures the following where selected: 
 	 {'50'}
	 FID 50:  Standing height
       50-0.0 50-1.0 50-2.0
0         154   <NA>   <NA>
1         174    172    173
2         166   <NA>    166
3         179   <NA>   <NA>
4         168   <NA>   <NA>
...       ...    ...    ...
502522    177   <NA>   <NA>
502523    161   <NA>   <NA>
502524    178   <NA>   <NA>
502525    175   <NA>   <NA>
502526    166   <NA>   <NA>

[502527 rows x 3 columns]


In [131]:
FID='21001'
#data_coding='100261'
FID_tidy_name="BMI"
selected_categories={fieldID_to_ctgry_dict[FID][1]:['include',FID]}
process_config={FID:{'type':'string'}}

df_temp=preprocess(selected_categories,categories,UID_to_dscrpt,df,process_config)

print(df_temp)
df_temp=df_temp[[FID+'-0.0']]

#df_tidy=tidy_coded_fieldID(bs,"Coding "+data_coding,FID_tidy_name+"_",FID,df_temp)

newdf=pd.concat([newdf,df_temp],axis=1)
df_tidy=df_temp

selected_categories_tidy[fieldID_to_ctgry_dict[FID][1]].append(FID_tidy_name)
selected_categories_tidy_columns[FID_tidy_name]={"col_name":list(df_tidy.columns.values),"type":"float"}

From the category Body size measures the following where selected: 
 	 {'21001'}
	 FID 21001:  Body mass index (BMI)
       21001-0.0 21001-1.0 21001-2.0
0        29.9798      <NA>      <NA>
1        24.4088   23.5938   22.5868
2        29.3584      <NA>   37.5236
3         30.617      <NA>      <NA>
4        31.3209      <NA>      <NA>
...          ...       ...       ...
502522   26.5888      <NA>      <NA>
502523   21.1797      <NA>      <NA>
502524   34.3707      <NA>      <NA>
502525   31.6082      <NA>      <NA>
502526   27.4713      <NA>      <NA>

[502527 rows x 3 columns]


In [132]:
FID='4080'
#data_coding='100261'
FID_tidy_name="SBP"
selected_categories={fieldID_to_ctgry_dict[FID][1]:['include',FID]}
process_config={FID:{'type':'string'}}

df_temp=preprocess(selected_categories,categories,UID_to_dscrpt,df,process_config)

df_temp=df_temp[[FID+'-0.0',FID+'-0.1']]

print('the percentage of na')
print(df_temp.isna().sum()/df_temp.shape[0])


df_temp[FID+'-0.0']= pd.to_numeric(df_temp[FID+'-0.0'], errors='coerce',downcast="float")
df_temp[FID+'-0.1']= pd.to_numeric(df_temp[FID+'-0.1'], errors='coerce',downcast="float")

#df_temp.fillna(df_temp.mean(),inplace=True)

df_temp['SBP_avg']=df_temp.mean(axis=1)
df_temp.drop(columns=[FID+'-0.0',FID+'-0.1'],inplace=True)

df_temp=df_temp.round(0)

df_temp
#df_tidy=tidy_coded_fieldID(bs,"Coding "+data_coding,FID_tidy_name+"_",FID,df_temp)

newdf=pd.concat([newdf,df_temp],axis=1)
df_tidy=df_temp

selected_categories_tidy[fieldID_to_ctgry_dict[FID][1]]=[FID_tidy_name]
selected_categories_tidy_columns[FID_tidy_name]={"col_name":list(df_tidy.columns.values),"type":"int"}

From the category Blood pressure the following where selected: 
 	 {'4080'}
	 FID 4080:  Systolic blood pressure, automated reading
the percentage of na
4080-0.0    0.068542
4080-0.1    0.082023
dtype: float64


In [133]:
FID='4079'
#data_coding='100261'
FID_tidy_name="DBP"
selected_categories={fieldID_to_ctgry_dict[FID][1]:['include',FID]}
process_config={FID:{'type':'string'}}

df_temp=preprocess(selected_categories,categories,UID_to_dscrpt,df,process_config)


df_temp=df_temp[[FID+'-0.0',FID+'-0.1']]

print('the percentage of na')
print(df_temp.isna().sum()/df_temp.shape[0])


df_temp[FID+'-0.0']= pd.to_numeric(df_temp[FID+'-0.0'], errors='coerce',downcast="float")
df_temp[FID+'-0.1']= pd.to_numeric(df_temp[FID+'-0.1'], errors='coerce',downcast="float")

#df_temp.fillna(df_temp.mean(),inplace=True)

df_temp['DBP_avg']=df_temp.mean(axis=1)
df_temp.drop(columns=[FID+'-0.0',FID+'-0.1'],inplace=True)

df_temp=df_temp.round(0)

df_temp
#df_tidy=tidy_coded_fieldID(bs,"Coding "+data_coding,FID_tidy_name+"_",FID,df_temp)

newdf=pd.concat([newdf,df_temp],axis=1)
df_tidy=df_temp

selected_categories_tidy[fieldID_to_ctgry_dict[FID][1]].append(FID_tidy_name)
selected_categories_tidy_columns[FID_tidy_name]={"col_name":list(df_tidy.columns.values),"type":"int"}

From the category Blood pressure the following where selected: 
 	 {'4079'}
	 FID 4079:  Diastolic blood pressure, automated reading
the percentage of na
4079-0.0    0.068516
4079-0.1    0.082015
dtype: float64


In [134]:
FID='102'
#data_coding='100261'
FID_tidy_name="PR"
selected_categories={fieldID_to_ctgry_dict[FID][1]:['include',FID]}
process_config={FID:{'type':'string'}}

df_temp=preprocess(selected_categories,categories,UID_to_dscrpt,df,process_config)

print(df_temp)
df_temp=df_temp[[FID+'-0.0',FID+'-0.1']]

print('the percentage of na')
print(df_temp.isna().sum()/df_temp.shape[0])


df_temp[FID+'-0.0']= pd.to_numeric(df_temp[FID+'-0.0'], errors='coerce',downcast="float")
df_temp[FID+'-0.1']= pd.to_numeric(df_temp[FID+'-0.1'], errors='coerce',downcast="float")

#df_temp.fillna(df_temp.mean(),inplace=True)

df_temp['PR_avg']=df_temp.mean(axis=1)
df_temp.drop(columns=[FID+'-0.0',FID+'-0.1'],inplace=True)

df_temp=df_temp.round(0)

df_temp
#df_tidy=tidy_coded_fieldID(bs,"Coding "+data_coding,FID_tidy_name+"_",FID,df_temp)

newdf=pd.concat([newdf,df_temp],axis=1)
df_tidy=df_temp

selected_categories_tidy[fieldID_to_ctgry_dict[FID][1]].append(FID_tidy_name)
selected_categories_tidy_columns[FID_tidy_name]={"col_name":list(df_tidy.columns.values),"type":"int"}

From the category Blood pressure the following where selected: 
 	 {'102'}
	 FID 102:  Pulse rate, automated reading
       102-0.0 102-0.1 102-1.0 102-1.1 102-2.0 102-2.1
0           83      82    <NA>    <NA>    <NA>    <NA>
1           67      64      53      55    <NA>    <NA>
2           57      60    <NA>    <NA>    <NA>    <NA>
3           57      56    <NA>    <NA>    <NA>    <NA>
4         <NA>    <NA>    <NA>    <NA>    <NA>    <NA>
...        ...     ...     ...     ...     ...     ...
502522      84      68    <NA>    <NA>    <NA>    <NA>
502523      57      59    <NA>    <NA>    <NA>    <NA>
502524      62      63    <NA>    <NA>    <NA>    <NA>
502525      75      74    <NA>    <NA>    <NA>    <NA>
502526      80      84    <NA>    <NA>    <NA>    <NA>

[502527 rows x 6 columns]
the percentage of na
102-0.0    0.068516
102-0.1    0.082015
dtype: float64


In [135]:
FID='30690'

FID_tidy_name="Total_Cholesterol"
selected_categories={fieldID_to_ctgry_dict[FID][1]:['include',FID]}
process_config={FID:{'type':'string'}}
df_temp=preprocess(selected_categories,categories,UID_to_dscrpt,df,process_config)


FID1='30760'
FID_tidy_name="HDL_Cholesterol"
selected_categories={fieldID_to_ctgry_dict[FID1][1]:['include',FID1]}
process_config={FID1:{'type':'string'}}
df_temp1=preprocess(selected_categories,categories,UID_to_dscrpt,df,process_config)

print(df_temp)
df_temp=df_temp[[FID+'-0.0']]
df_temp1=df_temp1[[FID1+'-0.0']]

print('the percentage of na')
print(df_temp.isna().sum()/df_temp.shape[0])
print(df_temp1.isna().sum()/df_temp1.shape[0])

FID_tidy_name="Cholesterol_to_HDL_Cholesterol"
df_tidy=pd.DataFrame(pd.to_numeric(df_temp["30690-0.0"],errors='coerce')/pd.to_numeric(df_temp1["30760-0.0"],errors='coerce'),columns=[FID_tidy_name],index=newdf.index)

newdf=pd.concat([newdf,df_tidy],axis=1)

selected_categories_tidy[fieldID_to_ctgry_dict[FID][1]]=[FID_tidy_name]
selected_categories_tidy_columns[FID_tidy_name]={"col_name":list(df_tidy.columns.values),"type":"binary"}

From the category Blood biochemistry the following where selected: 
 	 {'30690'}
	 FID 30690:  Cholesterol
From the category Blood biochemistry the following where selected: 
 	 {'30760'}
	 FID 30760:  HDL cholesterol
       30690-0.0 30690-1.0
0          5.847      <NA>
1          5.652     6.145
2          5.977      <NA>
3          7.548      <NA>
4           5.17      <NA>
...          ...       ...
502522     5.059      <NA>
502523     5.867      <NA>
502524     5.667      <NA>
502525     6.642      <NA>
502526     3.946      <NA>

[502527 rows x 2 columns]
the percentage of na
30690-0.0    0.065503
dtype: float64
30760-0.0    0.144545
dtype: float64


In [136]:
FID='20117'
data_coding='90'
FID_tidy_name="Alcohol Status"
selected_categories={fieldID_to_ctgry_dict[FID][1]:['include',FID]}
process_config={FID:{'type':'string'}}

df_temp=preprocess(selected_categories,categories,UID_to_dscrpt,df,process_config)

print(df_temp)
df_temp=df_temp[[FID+'-0.0']]

print('the percentage of na')
print(df_temp.isna().sum()/df_temp.shape[0])

df_tidy=tidy_coded_fieldID(bs,"Coding "+data_coding,FID_tidy_name+"_",FID,df_temp)

newdf=pd.concat([newdf,df_tidy],axis=1)

selected_categories_tidy[fieldID_to_ctgry_dict[FID][1]]=[FID_tidy_name]
selected_categories_tidy_columns[FID_tidy_name]={"col_name":list(df_tidy.columns.values),"type":"binary"}

From the category Alcohol the following where selected: 
 	 {'20117'}
	 FID 20117:  Alcohol drinker statusUses data-coding 90
       20117-0.0 20117-1.0 20117-2.0
0              2      <NA>      <NA>
1              2         2         2
2              2      <NA>         2
3              2      <NA>      <NA>
4              2      <NA>      <NA>
...          ...       ...       ...
502522         2      <NA>      <NA>
502523         2      <NA>      <NA>
502524         2      <NA>      <NA>
502525         2      <NA>      <NA>
502526         0      <NA>      <NA>

[502527 rows x 3 columns]
the percentage of na
20117-0.0    0.001785
dtype: float64
Alcohol Status_Current                 460379
Alcohol Status_Never                    22387
Alcohol Status_Prefer not to answer       757
Alcohol Status_Previous                 18107
dtype: int64


In [137]:
FID='20116'
data_coding='90'
FID_tidy_name="Smoking Status"
selected_categories={fieldID_to_ctgry_dict[FID][1]:['include',FID]}
process_config={FID:{'type':'string'}}

df_temp=preprocess(selected_categories,categories,UID_to_dscrpt,df,process_config)

df_temp=df_temp[[FID+'-0.0']]

print('the percentage of na')
print(df_temp.isna().sum()/df_temp.shape[0])

df_tidy=tidy_coded_fieldID(bs,"Coding "+data_coding,FID_tidy_name+"_",FID,df_temp)

newdf=pd.concat([newdf,df_tidy],axis=1)

selected_categories_tidy[fieldID_to_ctgry_dict[FID][1]]=[FID_tidy_name]
selected_categories_tidy_columns[FID_tidy_name]={"col_name":list(df_tidy.columns.values),"type":"binary"}

From the category Smoking the following where selected: 
 	 {'20116'}
	 FID 20116:  Smoking statusUses data-coding 90
the percentage of na
20116-0.0    0.001773
dtype: float64
Smoking Status_Current                  52979
Smoking Status_Never                   273533
Smoking Status_Prefer not to answer      2059
Smoking Status_Previous                173065
dtype: int64


In [138]:
newdf.to_csv(r'tidy_data_tiles/Anthropometrics and cardiac risk factors.csv')

# 2.4 Baseline medical conditions

## 2.4.1 Read data-coding 6 from the html file to process 20002 (self-reported non-cancer illnesses)

In [139]:
newdf=pd.DataFrame()

## Since -99999 and -1 entries cause problems this does not use the function tidy_coded_fieldID()

In [140]:
FID='20002'  #20009 is the corresponding age
data_coding='6'
FID_tidy_name="Baseline Medical conditions"

selected_categories={fieldID_to_ctgry_dict[FID][1]:['include',FID]}
process_config={FID:{'type':'string'}}

df_temp=preprocess(selected_categories,categories,UID_to_dscrpt,df,process_config)

#df_tidy=tidy_coded_fieldID(bs,"Coding "+data_coding,FID_tidy_name+"_",FID,df_temp)

From the category Medical conditions the following where selected: 
 	 {'20002'}
	 FID 20002:  Non-cancer illness code, self-reportedUses data-coding 6


In [141]:
html_file = open(PATH_dict)
html_content = html_file.read()

bs = BeautifulSoup(html_content)
table = bs.find(lambda tag: tag.name=='table' and tag.has_attr('summary') and tag['summary']=="Coding 6") 

pd_coding=pd.read_html(str(table))[0].astype(str)

In [142]:
newdf=df_temp
#Read all diseases (all instances) and compare times with recruitment time. if it's before age at recruitment consider it as baseline disease otherwise it's and endline disease
disease_code_to_disease_dict=pd_coding[['Code','Meaning']].set_index('Code').to_dict()['Meaning']
#disease_code_to_disease_dict['-1'] = disease_code_to_disease_dict.pop('-1(not selectable)')
#disease_code_to_disease_dict['99999'] = disease_code_to_disease_dict.pop('99999(not selectable)')
disease_code_to_disease_dict['-1']='deleteASAP'
disease_code_to_disease_dict['99999']='deleteASAP'
#disease_code_to_disease_dict_time=[value+'_t' for value in disease_code_to_disease_dict.values()]
col_names=list(disease_code_to_disease_dict.values())#+disease_code_to_disease_dict_time
col_names.sort()
df_tidy=pd.DataFrame(index=newdf.index,columns=col_names)


In [143]:
FID='20002'
#grab columns corresponding to the first assessment x-0.y
for patientID in newdf.index:
    cols_FID=[col for col in newdf.columns if (FID in col) and col.split('-')[1].split('.')[0]=='0']
    for col in cols_FID:
        entry=newdf.loc[patientID,col]
        if not pd.isna(entry):
            df_tidy.loc[patientID,disease_code_to_disease_dict[entry]]=1
df_tidy.fillna(0,inplace=True)
df_tidy.drop(columns=['deleteASAP'],inplace=True)

In [144]:
df_tidy.sum()

abdominal hernia                                        325
abnormal smear (cervix)                                 701
acne/acne vulgaris                                      298
acromegaly                                               66
acute infective polyneuritis/guillain-barre syndrome    195
                                                       ... 
vocal cord polyp                                        200
wegners granulmatosis                                    94
whooping cough / pertussis                              510
wolff parkinson white / wpw syndrome                     79
yellow fever                                             11
Length: 445, dtype: int64

In [145]:
word='ang'
for col in df_tidy.columns:
    if word in col.lower():
        print(col)

angina
bile duct obstruction/ascending cholangitis
sclerosing cholangitis


### Exclusions

disease exclusion: 
1. 'atrial fibrillation'
2. 'atrial flutter'
3. 'aortic valve disease' 
4. 'aortic regurgitation / incompetence',
5. 'aortic stenosis' 
*6. 'heart attack/myocardial infarction'
*7. 'heart failure/pulmonary odema'
*8. 'heart valve problem/heart murmur'
9. 'heart arrhythmia'
10. 'irregular heart beat'
11. 'hypertrophic cardiomyopathy (hcm / hocm)'
*12. 'heart/cardiac problem'
13. 'cardiomyopathy'
14. 'myocarditis'
*15. 'ischaemic stroke'
16. 'mitral regurgitation / incompetence'
17. 'mitral stenosis'
18. 'mitral valve disease'
19. 'mitral valve prolapse'
20. 'svt / supraventricular tachycardia'


How about:
1. 'aortic aneurysm',
2. 'aortic aneurysm rupture'
3. 'aortic dissection'
7. 'arterial embolism'  
*9. 'peripheral vascular disease'
11. 'varicose ulcer'
12. 'varicose veins'
13. 'venous thromboembolic disease'
14. deep venous thrombosis (dvt)
15. pericardial effusion
16. pericardial problem
17. pericarditis
microscopic polyarteritis
polyartertis nodosa

predictor:
1. 'hypertension'


"""

In [146]:
exclude_self_reported=['atrial fibrillation',
'angina',
'atrial flutter',
'aortic valve disease',
'aortic regurgitation / incompetence',
'aortic stenosis',
'heart attack/myocardial infarction',
'heart failure/pulmonary odema',
'heart valve problem/heart murmur',
'heart arrhythmia',
'irregular heart beat',
'hypertrophic cardiomyopathy (hcm / hocm)',
'heart/cardiac problem',
'cardiomyopathy',
'myocarditis',
'ischaemic stroke',
'mitral regurgitation / incompetence',
'mitral stenosis',
'mitral valve disease',
'mitral valve prolapse',
'svt / supraventricular tachycardia']

In [147]:
df_tidy['Exclude_baseline_MedCon']=df_tidy[exclude_self_reported].sum(axis=1)
df_tidy.drop(columns=exclude_self_reported,inplace=True)

In [148]:
newdf=df_tidy

selected_categories_tidy[fieldID_to_ctgry_dict[FID][1]]=[FID_tidy_name]
selected_categories_tidy_columns[FID_tidy_name]={"col_name":list(df_tidy.columns.values),"type":"binary"}
selected_categories_exclusions.append('Exclude_baseline_MedCon')

In [149]:
newdf.to_csv(r'tidy_data_tiles/Baseline Medical Conditions.csv')

## QRISK parameters from self-reported

In [150]:
qrisk_df=pd.DataFrame(columns=["diabetes"],index=newdf.index)

### diabetes SR

In [151]:
pat=re.compile('diab')
cols=list(filter(pat.findall, newdf.columns.values))
cols

['diabetes',
 'diabetes insipidus',
 'diabetic eye disease',
 'diabetic nephropathy',
 'diabetic neuropathy/ulcers',
 'gestational diabetes',
 'type 1 diabetes',
 'type 2 diabetes']

In [152]:
cols=['diabetes',
 'diabetic eye disease',
 'diabetic nephropathy',
 'diabetic neuropathy/ulcers',
 'type 1 diabetes',
 'type 2 diabetes',
 'kidney nephropathy']

In [153]:
newdf[cols].sum()

diabetes                      21724
diabetic eye disease           1171
diabetic nephropathy             20
diabetic neuropathy/ulcers      153
type 1 diabetes                 438
type 2 diabetes                3377
kidney nephropathy               70
dtype: int64

In [154]:
qrisk_df["diabetes_1"]=newdf['type 1 diabetes']

In [155]:
qrisk_df["diabetes_2"]=newdf['type 2 diabetes']

### Chronic Kidney disease
Only failure was reported 

In [156]:
pat=re.compile('beat')
cols=list(filter(pat.findall, newdf.columns.values))
cols

[]

In [157]:
pat=re.compile('renal')
cols2=list(filter(pat.findall, newdf.columns.values))
cols2

['adrenal tumour',
 'disorder of adrenal gland',
 'other renal/kidney problem',
 'renal failure not requiring dialysis',
 'renal failure requiring dialysis',
 'renal/kidney failure']

In [158]:
cols=['renal failure not requiring dialysis','renal failure requiring dialysis','renal/kidney failure']
newdf[cols].sum()

renal failure not requiring dialysis    398
renal failure requiring dialysis        267
renal/kidney failure                    232
dtype: int64

In [159]:
disease_name="CKD"
qrisk_df[disease_name]=0

## Migraine 

In [160]:
pat=re.compile('migraine')
cols=list(filter(pat.findall, newdf.columns.values))
cols

['headaches (not migraine)', 'migraine']

In [161]:
cols=['migraine']
newdf[cols].sum()

migraine    14384
dtype: int64

In [162]:
disease_name="Migraine"
qrisk_df[disease_name]=newdf[cols]

## Rheumatoid arthritis

In [163]:
pat=re.compile('arthritis')
cols=list(filter(pat.findall, newdf.columns.values))
cols

['arthritis (nos)',
 'osteoarthritis',
 'rheumatoid arthritis',
 'spine arthritis/spondylitis']

In [164]:
cols=['rheumatoid arthritis']
newdf[cols].sum()

rheumatoid arthritis    5658
dtype: int64

In [165]:
disease_name="Rheumatoid arthritis"
qrisk_df[disease_name]=newdf[cols]

## systemic lupus erythematosis/sle

In [166]:
pat=re.compile('systemic lupus erythematosis/sle')
cols=list(filter(pat.findall, newdf.columns.values))
cols

['systemic lupus erythematosis/sle']

In [167]:
cols=['systemic lupus erythematosis/sle']
newdf[cols].sum()

systemic lupus erythematosis/sle    642
dtype: int64

In [168]:
disease_name="SLE"
qrisk_df[disease_name]=newdf[cols]

## Severe mental illness

In [169]:
pat=re.compile('dep')
cols=list(filter(pat.findall, newdf.columns.values))
cols

['alcohol dependency',
 'depression',
 'mania/bipolar disorder/manic depression',
 'opioid dependency',
 'other substance abuse/dependency',
 'post-natal depression']

In [170]:
cols=['mania/bipolar disorder/manic depression',"schizophrenia"]
newdf[cols].sum()

mania/bipolar disorder/manic depression    1411
schizophrenia                               618
dtype: int64

In [171]:
disease_name="severe_mental"
qrisk_df.loc[newdf[cols].any(axis=1),disease_name]=1

## Erectile dysfunction

In [172]:
pat=re.compile('erect')
cols=list(filter(pat.findall, newdf.columns.values))
cols

['erectile dysfunction / impotence']

In [173]:
cols=['erectile dysfunction / impotence']
newdf[cols].sum()

erectile dysfunction / impotence    305
dtype: int64

In [174]:
disease_name="erectile_dysfunction"
qrisk_df.loc[newdf[cols].any(axis=1),disease_name]=1

In [175]:
qrisk_df.sum()

diabetes                    0.0
diabetes_1                438.0
diabetes_2               3377.0
CKD                         0.0
Migraine                14384.0
Rheumatoid arthritis     5658.0
SLE                       642.0
severe_mental            1995.0
erectile_dysfunction      305.0
dtype: float64

### Hypertension

In [176]:
pat=re.compile('hypertension')
cols=list(filter(pat.findall, newdf.columns.values))
cols

['essential hypertension',
 'gestational hypertension/pre-eclampsia',
 'hypertension']

# 2.5  Home Location

In [177]:
FID='22700'  #53 is the corresponding age
FID_tidy_name="Home location EC"

selected_categories={fieldID_to_ctgry_dict[FID][1]:['include',FID]}
process_config={FID:{'type':'string'}}

df_temp=preprocess(selected_categories,categories,UID_to_dscrpt,df,process_config).apply(pd.to_datetime)

for index in df_temp.index:
    df_temp.iloc[index]
    

#Get date first attended
selected_categories1={fieldID_to_ctgry_dict['53'][1]:['include','53']}
process_config1={'53':{'type':'string'}}

df_date_assessment=pd.to_datetime(preprocess(selected_categories1,categories,
                                             UID_to_dscrpt,df,process_config1)['53-0.0'],format='%Y-%m-%d').to_frame()

#df_tidy=tidy_coded_fieldID(bs,"Coding "+data_coding,FID_tidy_name+"_",FID,df_temp)

From the category Home locations the following where selected: 
 	 {'22700'}
	 FID 22700:  Date first recorded at locationUses data-coding 1313
From the category Reception the following where selected: 
 	 {'53'}
	 FID 53:  Date of attending assessment centre


In [178]:
df_valid=pd.DataFrame(index=df_temp.index,columns=['Exclude_Home_location','Home_Longitude','Home_Latitude'])
for index in df_temp.index:
    df_valid.loc[index,'Exclude_Home_location']=any((df_temp.loc[index,df_temp.iloc[index].notna()] > df_date_assessment.loc[index,'53-0.0']) & (df_temp.loc[index,df_temp.iloc[index].notna()]< df_date_assessment.loc[index,'53-0.0']+pd.Timedelta(weeks=261)))#checks if the last date for which I have a registered address is smaller than the date of assessment. If this is not true it means after the assessment the address has changes and the patient should be excluded
df_valid['Home_Longitude']=df['20074-0.0']
df_valid['Home_Latitude']=df['20075-0.0']

In [179]:
df_valid.shape

(502527, 3)

In [180]:
selected_categories_tidy['Home locations']=['Home location at assessment']
selected_categories_tidy_columns['Home location at assessment']={"col_name":['Home_Longitude','Home_Latitude'],"type":"int"}
selected_categories_exclusions.append("Exclude_Home_location")

## 2.5.1 Add 20118 _ Home area population density

In [181]:
FID='20118'  #53 is the corresponding age
FID_tidy_name="Home area population density"

selected_categories={fieldID_to_ctgry_dict[FID][1]:['include',FID]}
process_config={FID:{'type':'string'}}

df_temp=preprocess(selected_categories,categories,UID_to_dscrpt,df,process_config)

From the category Reception the following where selected: 
 	 {'20118'}
	 FID 20118:  Home area population density - urban or ruralUses data-coding 91


In [182]:
newdf=pd.concat([df_valid,df_temp],axis=1)

selected_categories_tidy['Home locations'].append(FID_tidy_name)
selected_categories_tidy_columns[FID_tidy_name]={"col_name":list(df_temp.columns.values),"type":"int"}

In [183]:
newdf.to_csv(r'tidy_data_tiles/Home locations.csv')

# 2.6 Pollutants

## 2.6.1 Pollutants

In [184]:
#PM 24005  24006 24007 24008   NOx 24003  24004  Noise   24020 24021 24022

In [185]:
selected_categories={'Residential air pollution':['include','24003', '24004','24005',  '24006', '24007', '24008']}
process_config={'24005':{'type':'string'},'24006':{'type':'string'},'24007':{'type':'string'},'24008':{'type':'string'},
                '24003':{'type':'string'},'24004':{'type':'string'}}

df_temp=preprocess(selected_categories,categories,UID_to_dscrpt,df,process_config)

print(df_temp)

newdf=df_temp

selected_categories_tidy_temp={'Residential air pollution':['24003: Nitrogen dioxide air pollution; 2010',
 '24004: Nitrogen oxides air pollution; 2010',
 '24005: Particulate matter air pollution (pm10); 2010',
 '24006: Particulate matter air pollution (pm2.5); 2010',
 '24007: Particulate matter air pollution (pm2.5) absorbance; 2010',
 '24008: Particulate matter air pollution 2.5-10um; 2010']}
selected_categories_tidy_columns_temp={'24003: Nitrogen dioxide air pollution; 2010':{"col_name":['24003-0.0'],"type":'float'},
                                       '24004: Nitrogen oxides air pollution; 2010':{"col_name":['24004-0.0'],"type":'float'},
                                       '24005: Particulate matter air pollution (pm10); 2010':{"col_name":['24005-0.0'],"type":'float'},
                                       '24006: Particulate matter air pollution (pm2.5); 2010':{"col_name":['24006-0.0'],"type":'float'},
                                       '24007: Particulate matter air pollution (pm2.5) absorbance; 2010':{"col_name":['24007-0.0'],"type":'float'},
                                       '24008: Particulate matter air pollution 2.5-10um; 2010':{"col_name":['24008-0.0'],"type":'float'}}


From the category Residential air pollution the following where selected: 
 	 {'24005', '24007', '24003', '24008', '24006', '24004'}
	 FID 24005:  Particulate matter air pollution (pm10); 2010
	 FID 24007:  Particulate matter air pollution (pm2.5) absorbance; 2010
	 FID 24003:  Nitrogen dioxide air pollution; 2010
	 FID 24008:  Particulate matter air pollution 2.5-10um; 2010
	 FID 24006:  Particulate matter air pollution (pm2.5); 2010
	 FID 24004:  Nitrogen oxides air pollution; 2010
       24005-0.0 24007-0.0 24003-0.0 24008-0.0          24006-0.0 24004-0.0
0          14.48      1.23     31.57      5.68               10.4     53.83
1          15.93      1.26     29.36      6.02  9.960000000000001     45.93
2          16.84       1.3     29.68      6.42               9.27     46.93
3          19.17      1.58      33.7      7.79               11.6     60.85
4           14.9      0.97     25.46      5.87              10.85     45.61
...          ...       ...       ...       ...         

## 2.6.2 Road 
24009: Traffic intensity on the nearest road',
 '24010: Inverse distance to the nearest road',
 '24011: Traffic intensity on the nearest major road',
 '24012: Inverse distance to the nearest major road',
 '24013: Total traffic load on major roads',
 '24014: Close to major roadUses data-coding 7 comprises 2 Integer-valued members in a simple list.',
 '24015: Sum of road length of major roads within 100m',

In [186]:
selected_categories={'Residential air pollution':['include','24009',  '24010', '24011', '24012',
                                                 '24013', '24014','24015']}
process_config={'24009':{'type':'string'},'24010':{'type':'string'},'24011':{'type':'string'},'24012':{'type':'string'},
                '24013':{'type':'string'},'24014':{'type':'string'},'24015':{'type':'string'}}
df_temp=preprocess(selected_categories,categories,UID_to_dscrpt,df,process_config)

From the category Residential air pollution the following where selected: 
 	 {'24010', '24012', '24015', '24011', '24009', '24014', '24013'}
	 FID 24010:  Inverse distance to the nearest road
	 FID 24012:  Inverse distance to the nearest major road
	 FID 24015:  Sum of road length of major roads within 100m
	 FID 24011:  Traffic intensity on the nearest major road
	 FID 24009:  Traffic intensity on the nearest road
	 FID 24014:  Close to major roadUses data-coding 12
	 FID 24013:  Total traffic load on major roads


In [187]:
newdf=pd.concat([newdf,df_temp],axis=1)

In [188]:
newdf['24014-0.0'].fillna('0',inplace=True)

In [189]:
selected_categories_tidy_temp['Residential air pollution']+=[ '24009: Traffic intensity on the nearest road',
 '24010: Inverse distance to the nearest road',
 '24011: Traffic intensity on the nearest major road',
 '24012: Inverse distance to the nearest major road',
 '24013: Total traffic load on major roads',
 '24014: Close to major roadUses data-coding 7 comprises 2 Integer-valued members in a simple list.',
 '24015: Sum of road length of major roads within 100m']
selected_categories_tidy_columns_temp1={'24009: Traffic intensity on the nearest road':{"col_name":['24009-0.0'],"type":'int'},
                                       '24010: Inverse distance to the nearest road':{"col_name":['24010-0.0'],"type":'float'},
                                       '24011: Traffic intensity on the nearest major road':{"col_name":['24011-0.0'],"type":'int'},
                                       '24012: Inverse distance to the nearest major road':{"col_name":['24012-0.0'],"type":'float'},
                                       '24013: Total traffic load on major roads':{"col_name":['24013-0.0'],"type":'int'},
                                       '24014: Close to major road':{"col_name":['24014-0.0'],"type":'binary'},
                                       '24015: Sum of road length of major roads within 100m':{"col_name":['24015-0.0'],"type":'float'}}

selected_categories_tidy_columns_temp={**selected_categories_tidy_columns_temp,**selected_categories_tidy_columns_temp1}

In [190]:
selected_categories_tidy={**selected_categories_tidy,**selected_categories_tidy_temp}
selected_categories_tidy_columns={**selected_categories_tidy_columns,**selected_categories_tidy_columns_temp}

In [191]:
newdf.to_csv(r'tidy_data_tiles/Pollutants.csv')

In [192]:
def save_obj(obj, name ):
    with open('obj_'+ name + '.pkl', 'wb') as f:
        pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)

def load_obj(name ):
    with open('obj_' + name + '.pkl', 'rb') as f:
        return pickle.load(f)

In [193]:
import pickle
save_obj(selected_categories_tidy,'selected_categories_tidy')
save_obj(selected_categories_tidy_columns,'selected_categories_tidy_columns')
save_obj(selected_categories_exclusions,'selected_categories_exclusions')

In [194]:
load_obj('selected_categories_exclusions')

['Exclude_baseline_MedCon', 'Exclude_Home_location']

# 2.3 Read early life factors and store 

In [195]:
categories['Early life factors']

['120: Birth weight knownUses data-coding 100258',
 '129: Place of birth in UK - north co-ordinateUses data-coding 170',
 '130: Place of birth in UK - east co-ordinateUses data-coding 170',
 '1647: Country of birth (UK/elsewhere)Uses data-coding 100420',
 '1677: Breastfed as a babyUses data-coding 100349',
 '1687: Comparative body size at age 10Uses data-coding 100428',
 '1697: Comparative height size at age 10Uses data-coding 100429',
 '1707: Handedness (chirality/laterality)Uses data-coding 100430',
 '1767: Adopted as a childUses data-coding 100435',
 '1777: Part of a multiple birthUses data-coding 100435',
 '1787: Maternal smoking around birthUses data-coding 100435',
 '20022: Birth weight',
 '20115: Country of Birth (non-UK origin)Uses data-coding 89']

In [196]:
selected_categories={'Early life factors':['include','20022','129','130','20115']}
process_config={'20022':{'type':'float'},'129':{'type':'float'},'130':{'type':'float'},'20115':{'type':'float'}}
newdf=preprocess(selected_categories,categories,UID_to_dscrpt,df,process_config)

From the category Early life factors the following where selected: 
 	 {'130', '129', '20022', '20115'}
	 FID 130:  Place of birth in UK - east co-ordinateUses data-coding 170
	 FID 129:  Place of birth in UK - north co-ordinateUses data-coding 170
	 FID 20022:  Birth weight
	 FID 20115:  Country of Birth (non-UK origin)Uses data-coding 89


In [197]:
newdf=newdf[["129-0.0","130-0.0","20115-0.0","20022-0.0"]]

In [198]:
selected_categories_tidy_temp={'Early life factors':['Place of birth in UK - north co-ordinate','Place of birth in UK - east co-ordinate','Country of Birth','Birth weight']}
selected_categories_tidy_columns_temp={'Place of birth in UK - north co-ordinate':{"col_name":['129-0.0'],"type":"int"},
                                       'Place of birth in UK - east co-ordinate':{"col_name":['130-0.0'],"type":"int"},
                                       'Country of Birth':{"col_name":['20115-0.0'],"type":"int"},
                                       'Birth weight':{"col_name":['20022-0.0'],"type":"float"}}
newdf.to_csv(r'tidy_data_tiles/Early life factors.csv')

In [199]:
selected_categories_tidy={**selected_categories_tidy,**selected_categories_tidy_temp}
selected_categories_tidy_columns={**selected_categories_tidy_columns,**selected_categories_tidy_columns_temp}

## 2.4 Process 41270 & 41280 (primary and secondary)

In [205]:
word='ICD10'
ukb.search_description(word)

Field ID: 40001 _ Underlying (primary) cause of death: ICD10Uses data-coding 19

Field ID: 40002 _ Contributory (secondary) causes of death: ICD10Uses data-coding 19

Field ID: 40006 _ Type of cancer: ICD10Uses data-coding 19

Field ID: 41201 _ External causes - ICD10Uses data-coding 240

Field ID: 41202 _ Diagnoses - main ICD10Uses data-coding 240

Field ID: 41204 _ Diagnoses - secondary ICD10Uses data-coding 240

Field ID: 41262 _ Date of first in-patient diagnosis - main ICD10

Field ID: 41270 _ Diagnoses - ICD10Uses data-coding 259

Field ID: 41280 _ Date of first in-patient diagnosis - ICD10



In [206]:
df.loc[0:20,"41270-0.0"].unique()

array(['I849', nan, 'J181', 'K429', 'E780', 'C504', 'H186', 'D683',
       'A099', 'H024', 'I10', 'D237', 'N950', 'D170', 'N840', 'B949'],
      dtype=object)

In [207]:
print(ukb._UID_to_dscrpt['41270'])
print(ukb._fieldID_to_ctgry_dict['41270'])

[8046, 5865041, 'Categorical (multiple)', 'Diagnoses - ICD10Uses data-coding 259', 0, 212]
['Diagnoses - ICD10', 'Summary Diagnoses']


In [208]:
selected_categories={'Summary Diagnoses':['include','41270']}
process_config={'41270':{'type':'string'}}

df_temp=preprocess(selected_categories,ukb.categories,ukb._UID_to_dscrpt,df,process_config)


selected_categories={'Summary Diagnoses':['include','41280']}
process_config={'41280':{'type':'string'}}
df_temp_date=preprocess(selected_categories,ukb.categories,ukb._UID_to_dscrpt,df,process_config)

selected_categories1={fieldID_to_ctgry_dict['53'][1]:['include','53']}
process_config1={'53':{'type':'string'}}

df_date_assessment=pd.to_datetime(preprocess(selected_categories1,ukb.categories,
                                             ukb._UID_to_dscrpt,df,process_config1)['53-0.0'],format='%Y-%m-%d').to_frame()

df_temp_date=df_temp_date.apply(pd.to_datetime)


cols_dict={df_temp.columns.values[i]:df_temp_date.columns.values[i] for i in range(df_temp.shape[1])}
df_temp.rename(columns=cols_dict,inplace=True)

From the category Summary Diagnoses the following where selected: 
 	 {'41270'}
	 FID 41270:  Diagnoses - ICD10Uses data-coding 259
From the category Summary Diagnoses the following where selected: 
 	 {'41280'}
	 FID 41280:  Date of first in-patient diagnosis - ICD10
From the category Reception the following where selected: 
 	 {'53'}
	 FID 53:  Date of attending assessment centre


In [209]:
baseline_diagnoses=df_temp.where(df_temp_date.lt(df_date_assessment["53-0.0"],axis=0))
baseline_diagnoses_dates=df_temp_date.where(df_temp_date.lt(df_date_assessment["53-0.0"],axis=0))

In [210]:
baseline_diagnoses_sorted=pd.DataFrame(columns=baseline_diagnoses.columns, index=baseline_diagnoses.index)
baseline_diagnoses_dates_sorted=pd.DataFrame(columns=baseline_diagnoses.columns, index=baseline_diagnoses.index)

In [211]:
baseline_diagnoses_dates.max().max()

Timestamp('2010-09-14 00:00:00')

# This takes about 30 mins to run needs to be optimised

# Doesn't actually sort, but it's find for now as I will be working with BoW

In [212]:
tic=time.time()
for patient in baseline_diagnoses_dates.index:
    indxs_sorted=baseline_diagnoses_dates.iloc[patient].values.argsort()
    baseline_diagnoses_sorted.iloc[patient]=baseline_diagnoses.iloc[patient,indxs_sorted]
    baseline_diagnoses_dates_sorted.iloc[patient]=baseline_diagnoses_dates.iloc[patient,indxs_sorted]

toc=time.time()
print(toc-tic)

2262.580269098282


In [213]:
baseline_diagnoses_sorted.dropna(axis=1,how="all",inplace=True)

In [214]:
baseline_diagnoses_dates_sorted.dropna(axis=1,how="all",inplace=True)

In [215]:
baseline_diagnoses_sorted.head()

,41280-0.0,41280-0.1,41280-0.2,41280-0.3,41280-0.4,41280-0.5,41280-0.6,41280-0.7,41280-0.8,41280-0.9,...,41280-0.195,41280-0.196,41280-0.197,41280-0.198,41280-0.204,41280-0.205,41280-0.206,41280-0.208,41280-0.210,41280-0.211
0,I849,<NA>,<NA>,K621,Z858,<NA>,<NA>,<NA>,<NA>,<NA>,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
1,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
2,<NA>,<NA>,<NA>,<NA>,N62,<NA>,R074,<NA>,<NA>,<NA>,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
3,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
4,<NA>,E785,G560,<NA>,I10,I269,M1099,M170,M171,M2332,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>


In [216]:
def load_obj(name ):
    with open('obj_' + name + '.pkl', 'rb') as f:
        return pickle.load(f)
#1. read the dict to map each entry to column
entry_L3_to_column_dict=load_obj('ICD10_L3_simp_dict')
entry_L3_to_column_dict_dyno=load_obj('ICD10_L3_dyno_dict')

entry_L3_to_column_dict={k.replace(".",""):v for k,v in entry_L3_to_column_dict.items()}

In [217]:
entry_L2_to_column_dict=dict()
for k,v in entry_L3_to_column_dict_dyno.items():
    if v["jump_to"]=="level3":
         entry_L2_to_column_dict[v["level2_name"]]=v["level2_desc"]
    elif v["jump_to"] in ["chapter","level2","level1"]:
           entry_L2_to_column_dict[v["level2_name"]]=v[v["jump_to"]+"_desc"]
    else:
        print("what?!?!?1")

In [218]:
#2. define the the tidy data format
col_names=np.unique(list(entry_L3_to_column_dict.values()))#+disease_code_to_disease_dict_time
#col_names_time=[col_name+'_time' for col_name in col_names]
col_names_all=col_names#[*col_names, *col_names_time]
df_tidy_temp=pd.DataFrame(index=baseline_diagnoses_sorted.index,columns=col_names_all)

df_untidy_temp=baseline_diagnoses_sorted
df_untidy_temp_time=baseline_diagnoses_dates_sorted

In [219]:
df_tidy_temp.head()

,A04 Other bacterial intestinal infections,A08 Viral and other specified intestinal infections,A09.9 Gastroenteritis and colitis of unspecified origin,A30-A49 Other bacterial diseases,"A41.9 Septicaemia, unspecified",B00-B09 Viral infections characterized by skin and mucous membrane lesions,"B34.9 Viral infection, unspecified","C00-C14 Malignant neoplasms of lip, oral cavity and pharynx",C15 Malignant neoplasm of oesophagus,C15-C26 Malignant neoplasms of digestive organs,...,Z43.2 Attention to ileostomy,Z45.0 Adjustment and management of cardiac pacemaker,Z45.2 Adjustment and management of vascular access device,Z45.8 Adjustment and management of other implanted devices,Z46.6 Fitting and adjustment of urinary device,Z47.0 Follow-up care involving removal of fracture plate and other internal fixation device,Z48 Other surgical follow-up care,"Z53 Persons encountering health services for specifie procedures, not carried out",Z53.8 Procedure not carried out for other reasons,Z80-Z99 Persons with potential health hazards related to family and personal history and certain conditions influencing health status
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# This also takes around 3 hours, probably can be optimised
1. perhaps if I change the datatype from datetime
2. perhaps if I cut some of the unnecessary try excepts which I can count how many times they are activated

In [ ]:
tic=time.time()
for patientID in df_untidy_temp.index:
    if patientID%10000==0:
        print(patientID)
        toc=time.time()
        print(toc-tic)
    recorded_disease_indxs=df_untidy_temp.columns[df_untidy_temp.iloc[patientID].notnull()]
    for disease_indx in recorded_disease_indxs:
        entry=df_untidy_temp.loc[patientID,disease_indx]
        try:
            df_tidy_temp.loc[patientID,entry_L3_to_column_dict[entry]]=df_untidy_temp_time.loc[patientID,'41280-'+disease_indx.split('-')[1]]
        except:
            try:
                df_tidy_temp.loc[patientID,entry_L2_to_column_dict[entry]]=df_untidy_temp_time.loc[patientID,'41280-'+disease_indx.split('-')[1]]
            except:
                try:
                    df_tidy_temp.loc[patientID,entry_L3_to_column_dict[entry[:-1]]]=df_untidy_temp_time.loc[patientID,'41280-'+disease_indx.split('-')[1]]
                except:
                    try:
                        df_tidy_temp.loc[patientID,entry_L2_to_column_dict[entry[:-1]]]=df_untidy_temp_time.loc[patientID,'41280-'+disease_indx.split('-')[1]]
                    except:
                        try:
                            df_tidy_temp.loc[patientID,entry_L2_to_column_dict[entry[:-2]]]=df_untidy_temp_time.loc[patientID,'41280-'+disease_indx.split('-')[1]]
                        except:
                            pass
                            #print("error",patientID,entry)
                            #print("Should it happen?1")

toc=time.time()
print(toc-tic)

0
0.0012657642364501953
10000
208.28528761863708
20000
417.50253653526306
30000
627.7799670696259
40000
840.1352272033691
50000
1043.533392906189
60000
1250.27281665802
70000
1454.6214137077332
80000
1659.77791762352
90000
1869.9511225223541
100000
2077.3021228313446
110000
2287.2278294563293
120000
2498.56409907341
130000
2699.572174310684
140000
2899.488252401352
150000
3106.022352218628
160000
3313.6898186206818
170000
3514.4289247989655
180000
3726.519650220871
190000
3933.8888869285583
200000
4138.238597869873
210000
4346.353989124298
220000
4555.776926755905
230000
4764.032240629196
240000
4970.964054107666
250000
5177.690267562866
260000
5385.089805841446
270000
5588.521295785904
280000
5796.098433971405
290000
6003.397251605988
300000
6210.420321464539
310000
6418.598404169083
320000
6625.512753725052
330000
6829.2003610134125
340000
7034.002634763718
350000
7246.65044426918
360000
7450.038424015045


In [ ]:
selected_categories_tidy_temp={'Summary Diagnoses':['41270']}
selected_categories_tidy_columns_temp={'41270':{"col_name":df_tidy_temp.columns.values,"type":"date"}}
df_tidy_temp.to_csv(r'tidy_data_tiles/summary_diagnoses.csv')

In [ ]:
selected_categories_tidy={**selected_categories_tidy,**selected_categories_tidy_temp}
selected_categories_tidy_columns={**selected_categories_tidy_columns,**selected_categories_tidy_columns_temp}

# QRISK3
I have Age, sex, ethinicity, townedn deprivation indedx, smoking
I need,
1. diabetes, 

3.

Won't have:
1. angina or heart attach in a 1st degree<60 *

## 1. diabetes

In [ ]:

#find all who have a diagnoses
indxs_diag_T1=df_untidy_temp.loc[df_untidy_temp.apply(lambda col:col.str.contains('E10',na=False)).sum(axis=1)>0].index
indxs_diag_T2=df_untidy_temp.loc[df_untidy_temp.apply(lambda col:col.str.contains('E11',na=False)).sum(axis=1)>0].index

In [ ]:
print("T1: {T1:.0f} \n T2:{T2:.0f}".format(T1=len(indxs_diag_T1),T2=len(indxs_diag_T2)))

In [ ]:
disease_sr="diabetes_2"
indxs_diag=indxs_diag_T2

indxs_selfreported=qrisk_df[qrisk_df[disease_sr]==1].index

In [ ]:

in_both=len(set(indxs_selfreported).intersection(set(indxs_diag)))

print('self reported: {number_of_selfreported:.0f} \n \
diagnosed: {number_of_diagnosed:.0f} \n \
only in self reported: {only_in_sr:.0f} \n \
only in diagnosed: {only_in_d:.0f} \n \
in both: {in_both}'.format(number_of_selfreported=len(indxs_selfreported),
                           number_of_diagnosed=len(indxs_diag),
                           only_in_sr=len(indxs_selfreported)-in_both,
                           only_in_d=len(indxs_diag)-in_both,
                           in_both=in_both))


In [ ]:
qrisk_df.loc[indxs_diag_T1,"diabetes_1"]=1
qrisk_df.loc[indxs_diag_T2,"diabetes_2"]=1

##  2. CKD (3,4,5)

In [ ]:
indxs_diag=df_untidy_temp.loc[df_untidy_temp.apply(lambda col:col.str.contains('N18',na=False)).sum(axis=1)>0].index
print("num: {num:.0f}".format(num=len(indxs_diag)))

In [ ]:
XXX=df_untidy_temp.iloc[indxs_diag]
pd.unique(XXX.iloc[0])

In [ ]:
qrisk_df.loc[indxs_diag,"CKD"]=1

## 3. Atrial Fibrilation

In [ ]:
indxs_diag=df_untidy_temp.loc[df_untidy_temp.apply(lambda col:col.str.contains('I48',na=False)).sum(axis=1)>0].index
print("num: {num:.0f}".format(num=len(indxs_diag)))

In [ ]:
qrisk_df.loc[indxs_diag,"AF"]=1

## 4. Migraine

In [ ]:
indxs_diag=df_untidy_temp.loc[df_untidy_temp.apply(lambda col:col.str.contains('G43',na=False)).sum(axis=1)>0].index
print("num: {num:.0f}".format(num=len(indxs_diag)))

In [ ]:
qrisk_df.loc[indxs_diag,"Migraine"]=1

In [ ]:
qrisk_df.head()

## 5. Rheumatoid arthritis

In [ ]:
indxs_diag=df_untidy_temp.loc[df_untidy_temp.apply(lambda col:col.str.contains('J990|M05|M06',na=False)).sum(axis=1)>0].index

In [ ]:
disease_sr="Rheumatoid arthritis"
indxs_selfreported=qrisk_df[qrisk_df[disease_sr]==1].index


in_both=len(set(indxs_selfreported).intersection(set(indxs_diag)))

print('self reported: {number_of_selfreported:.0f} \n \
diagnosed: {number_of_diagnosed:.0f} \n \
only in self reported: {only_in_sr:.0f} \n \
only in diagnosed: {only_in_d:.0f} \n \
in both: {in_both}'.format(number_of_selfreported=len(indxs_selfreported),
                           number_of_diagnosed=len(indxs_diag),
                           only_in_sr=len(indxs_selfreported)-in_both,
                           only_in_d=len(indxs_diag)-in_both,
                           in_both=in_both))


In [ ]:
qrisk_df.loc[indxs_diag,"Rheumatoid arthritis"]=1

## 6.  systemic lupus erythematosis/sle

In [ ]:
indxs_diag=df_untidy_temp.loc[df_untidy_temp.apply(lambda col:col.str.contains('M32',na=False)).sum(axis=1)>0].index

In [ ]:
disease_sr="SLE"
indxs_selfreported=qrisk_df[qrisk_df[disease_sr]==1].index


in_both=len(set(indxs_selfreported).intersection(set(indxs_diag)))

print('self reported: {number_of_selfreported:.0f} \n \
diagnosed: {number_of_diagnosed:.0f} \n \
only in self reported: {only_in_sr:.0f} \n \
only in diagnosed: {only_in_d:.0f} \n \
in both: {in_both}'.format(number_of_selfreported=len(indxs_selfreported),
                           number_of_diagnosed=len(indxs_diag),
                           only_in_sr=len(indxs_selfreported)-in_both,
                           only_in_d=len(indxs_diag)-in_both,
                           in_both=in_both))


In [ ]:
qrisk_df.loc[indxs_diag,"SLE"]=1

## 7. Severe Mental Illness including psychosis, schizophrenia, or bipolar affective disease

In [ ]:
indxs_diag=df_untidy_temp.loc[df_untidy_temp.apply(lambda col:col.str.contains('F2|F30|F31|F32|F33',na=False)).sum(axis=1)>0].index

In [ ]:
len(indxs_diag)

In [ ]:
disease_sr="severe_mental"
indxs_selfreported=qrisk_df[qrisk_df[disease_sr]==1].index


in_both=len(set(indxs_selfreported).intersection(set(indxs_diag)))

print('self reported: {number_of_selfreported:.0f} \n \
diagnosed: {number_of_diagnosed:.0f} \n \
only in self reported: {only_in_sr:.0f} \n \
only in diagnosed: {only_in_d:.0f} \n \
in both: {in_both}'.format(number_of_selfreported=len(indxs_selfreported),
                           number_of_diagnosed=len(indxs_diag),
                           only_in_sr=len(indxs_selfreported)-in_both,
                           only_in_d=len(indxs_diag)-in_both,
                           in_both=in_both))


In [ ]:
qrisk_df.loc[indxs_diag,"severe_mental"]=1

## 8.Erectile disfunction  

In [ ]:
indxs_diag=df_untidy_temp.loc[df_untidy_temp.apply(lambda col:col.str.contains('F522|N484',na=False)).sum(axis=1)>0].index

In [ ]:
len(indxs_diag)

In [ ]:
disease_sr="erectile_dysfunction"
indxs_selfreported=qrisk_df[qrisk_df[disease_sr]==1].index


in_both=len(set(indxs_selfreported).intersection(set(indxs_diag)))

print('self reported: {number_of_selfreported:.0f} \n \
diagnosed: {number_of_diagnosed:.0f} \n \
only in self reported: {only_in_sr:.0f} \n \
only in diagnosed: {only_in_d:.0f} \n \
in both: {in_both}'.format(number_of_selfreported=len(indxs_selfreported),
                           number_of_diagnosed=len(indxs_diag),
                           only_in_sr=len(indxs_selfreported)-in_both,
                           only_in_d=len(indxs_diag)-in_both,
                           in_both=in_both))


In [ ]:
qrisk_df.loc[indxs_diag,"erectile_dysfunction"]=1

In [ ]:
qrisk_df.sum()

In [ ]:
qrisk_df.to_csv(r'tidy_data_tiles/qrisk.csv')

In [ ]:
import pickle
def save_obj(obj, name ):
    with open('obj_'+ name + '.pkl', 'wb') as f:
        pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)
save_obj(selected_categories_tidy,'selected_categories_tidy')
save_obj(selected_categories_tidy_columns,'selected_categories_tidy_columns')
save_obj(selected_categories_exclusions,'selected_categories_exclusions')